# Des biais et des erreurs communes

Les exercices suivants sont destinés à vous familiariser avec les concepts appréhendés lors de l’introduction au *machine learning*. Avant toute chose, importez les librairies utiles :

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from random import random

sns.set_context('notebook')

## Une intelligence artificielle drôlement intelligente

Prenons un jeu de données factice d’une centaine d’observations étiquetées pour les unes *cat*, pour les autres *dog*, avec des valeurs associées que l’on suppose dimensionnées :

In [ ]:
# 5, out of an hundred, are cats
series = {
    "label": [ "cat" if i < 5 else "dog" for i in range(100) ],
    "value": [ random() for i in range(100) ]
}

df = pd.DataFrame(series)

Afin d’éviter un biais évident relatif aux données non triées, nous les mélangeons grâce à une fonction de randomisation :

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

Notre objectif étant de programmer une machine qui va, pour une valeur de `X`, deviner si nous sommes en présence d’un chat ou d’un chien, il est temps d’extraire la variable cible du jeu de données :

In [ ]:
y = df.label
X = df.value

Nous pouvons maintenant programmer un prédicteur qui, pour chaque observation dans `X`, lui associera systématiquement l’étiquette *dog* :

In [ ]:
y_pred = [ "dog" for i in X ]

En comparant les prédictions `y_pred` avec la réalité (`y`), on peut calculer le taux d’exactitude :

In [ ]:
accuracy = (y_pred == y).sum()

print(f"Le taux d’exactitude (accuracy) de notre prédicteur est de { accuracy } % !")

Expliquez pourquoi un algorithme aussi bête que le nôtre a pu obtenir un score aussi étonnant.

## Les relations à distance

Il paraît que l’univers est en expansion et que cette expansion va en s’accélérant. C’est en tout cas ce que l’étude de Wendy Freedman et al. a prouvé ([*Freedman, 2001*](../0.about-datasets.ipynb#Stellar-Objects)). Par conséquent, on s’attend à ce qu’un objet stellaire s’éloigne d’autant plus vite de nous que la distance qui nous sépare de lui est grande.

Chargeons le jeu de données en se concentrant sur des objets proches de nous (entre 30 000 et 100 000 années-lumières) :

In [ ]:
# load data
df = pd.read_csv("../data/stellar-objects.csv", sep="\t")

# distance: megaparsec (MPC)
# velocity: in km/s
df["velocity"] = df.v_helio.fillna(df.v_flow.fillna(df.v_cmb))

# objects close to earth, but not that close :)
data = df[(df.distance > 10) & (df.distance < 30)]

Affichons un nuage de points afin de vérifier la proposition de ces pontes de la NASA :

In [ ]:
sns.scatterplot(data=data, x="distance", y="velocity")

sns.despine()

plt.title("Relation between distance and velocity of stellar objects")
plt.xlabel("Distance (MPC)")
plt.ylabel("Velocity (km/s)")

plt.show()

Euh… rien de bien concluant à première vue, non ? Afin de déterminer visuellement s’il existe bien une relation linéaire entre la distance et la vitesse d’éloignement, affichez une droite de régression :

In [ ]:
# your code here

_ = sns.regplot(data=data, x="distance", y="velocity")

Bon, appelez BFM TV, Wendy s’est trompée : 2/3 des points sont en dehors de l’intervalle de confiance à 95 %. Ou alors, peut-être avons-nous fait une erreur de méthodologie ?

## Une étude de genre

L’enquête [*Self-Reports of Height and Weight*](../0.about-datasets.ipynb#Self-Reports-of-Height-and-Weight) (Davis, 1990) compare une auto-évaluation de leurs tailles et poids d’individus engagés dans un programme d’exercices avec les mesures réalisées par l’équipe encadrante.

Imaginons un objectif où, en fonction des valeurs renseignées, on souhaiterait déduire l’étiquette *H* ou *F* qui leur est associée. Chargeons dans un premier temps les données et affichons un résumé :

In [ ]:
# load data
df = pd.read_csv("../data/davis.csv", sep="\t")

# select variables
target = "sex"
features = ["weight", "height", "repwt", "repht"]

# a copy of the data frame
data = df.copy()
data = data[[target] + features]

data.info()

Le jeu de données est composée de 200 observations mais comme toutes ne sont pas remplies pour tous les champs, il convient dans un premier temps de s’en occuper. Nous retenons comme stratégie de les combler avec la valeur moyenne de la colonne :

In [ ]:
# mean value
repwt_mean = int(data.repwt.mean())
repht_mean = int(data.repht.mean())

# fill NA
data.repwt.fillna(repwt_mean, inplace=True)
data.repht.fillna(repht_mean, inplace=True)

data.info()

La seconde étape consiste à séparer le *dataset* en deux parties inégales : l’une pour le jeu d’entraînement, constituée de 80 % de l’ensemble ; et l’autre pour le jeu de test.

In [ ]:
limit = int(len(data) * 0.2)

# split
train = data[limit:]
test = data[:limit]

Attachons-nous à étudier le rapport entre le poids et la taille des individus. Intuitivement, on penserait que ces caractéristiques sont globalement liées par une corrélation positive : l’augmentation chez l’une entraîne une augmentation chez l’autre. Si nous affichons une droite de régression sur le jeu de données complet, on observe bien le phénomène attendu :

In [ ]:
_ = sns.regplot(data=data, x="weight", y="height")

Pour autant, il n’en va pas de même avec les jeux d’entraînement et de test :

In [ ]:
figure, (col_1, col_2)= plt.subplots(ncols=2, figsize=(12,4))

sns.regplot(data=train, x="weight", y="height", ax=col_1)
sns.regplot(data=test, x="weight", y="height", ax=col_2)

figure.suptitle("Relation entre le poids et la taille des individus", y=1.05)

col_1.set(title="Jeu d’entraînement")
col_2.set(title="Jeu de test")

sns.despine()

plt.show()

À votre avis, quelles erreurs peuvent avoir faussé notre interprétation ?